#Variational Quantum Eigensolver(VQE)、QAOAセミナー 2-3（組合せ最適化問題編）

変分アルゴリズムの応用として組合せ最適化問題があります。多くの人に馴染みのある問題なので、量子コンピュータは正直どっから手をつけていいかわからんと思うひとおすすめです。

組合せ最適化問題は、多くの選択肢からベストな答えを選ぶ問題で、社会問題をうまく定式化し、最小値問題を解くことで最適な組合せを得ることができます。主に選択肢を選んだ状態を１、選ばない状態を0としますが、様々な条件をつけながら問題を解きます。

##2-4-1 例：VQEで組合せ最適化問題の定式化
定式化はハミルトニアンをコスト関数として扱い、最小値を返す量子ビットの組合せを解とします。ルールは、

・イジングモデルに落とし込む
・Zを使う
・最終的にはZの代わりにQUBOを使う

です。今回は通常VQEを組合せ最適化問題としてはあまり利用しませんが、例題のハミルトニアンを実行してみます。

```python
h = -Z(0) - Z(0)*Z(1)
```

Zの後ろの数字は、量子ビットの通し番号を表します。0番目と1番目の量子ビットの２つを使っています。また、問題設定で大事なのは、Zの前の係数です。

Z(0)の前は-1のバイアス
Z(0)*Z(1)の前は-1のウェイト

が設定されています。Zは期待値として-1か+1のどちらかをとります。hはより小さい値を取ると正解になります。最終的な答えは、Z(0),Z(1)の値で場合わけすると、

Z(0) | Z(1) | h
--:|:----:|:--
-1|-1|0
-1|1|2
1|-1|0
1|1|-2

VQEは上記の表で最小となるZ(0)=1,Z(1)=1の時-2となるものを計算で求めてくれます。今回ansatzはa,b,c,dの４パラメータを利用した極座標表記のものを使ってみます。ansatzを含む全体のコードは、

In [2]:
!pip3 install blueqat

     |████████████████████████████████| 51kB 1.6MB/s 


In [4]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.vqe import AnsatzBase, Vqe

class OneQubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian.to_expr(), 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]


# この定式化が大事
h = -Z(0) - Z(0)*Z(1)
runner = Vqe(OneQubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

Result by VQE
-1.99999508430852


約-2が答えとして出てきましたので、正解です。今回は例題として任意の量子状態を使いましたが、通常はQAOAを使います。そして実際にはより大きな問題を解きます。上記問題は役にたたなさそうですが、Z(0)をAさん、Z(1)をBさんに見立てて、Aさんはグループ１に属し、BさんはAさんと同じグループに属するという条件をつけた分類問題と同じです。

ただ、このままでは毎回問題を解くのが大変なので様々な工夫が必要です、それをみていきましょう。


##2-4-2 定式化は0と1のバイナリ値で
定式化は組合せ最適化問題を+1と-1の値の組合せで表される式に落とし込みます。ただ、課題があります。物理学で使われるイジングモデルは-1,+1を利用しますが、通常の産業用では0と1を計算として利用します（計算基底）。幸い-1と+1は自動的に変換ができますので、01を使っても組合せ最適化問題としては問題がありません。

01で定式化をするのをQUBOと呼びます。QUBOは量子ビットの01を利用できます。係数は様々な呼び方がありますが「バイアス」と「ウェイト（結合荷重）」と呼びます。定式化で作るのは「コスト関数」と呼びたいと思います。

-1と+1でかかれたイジング式を0と1で書かれたQUBO式に変換するには、イジングのZを下記のように変換するだけでできます。

$$
q = \frac{Z + 1}{2}
$$

これで、-1の時が0に、1の時は1のままで変換されます。定式化は社会問題をQUBO形式で、コスト関数を作ることで実現でき、コスト関数は01の値をとる量子ビットに設定するバイアスとウェイトを設定することで実現できます。この手法は量子コンピュータに限ったことではないので普通の計算に慣れている人でも受け入れられやすいでしょう。

##2-4-3 QUBO式のプログラミング
別の問題をQUBOで解いてみるため、blueqatの機能を利用します。QUBOへの変換はZを代入して書き換えればいいだけでした。

今回のときたいコスト関数は、

```python
h = -3*q(0)-3*q(1)-2*q(0)*q(1)
```

明らかにq(0)=1,q(1)=1の時に最小値-3-3-2=-8を取ります。上記のコスト関数は以前のように-1と+1ではなく、0と1で考えることができるので簡単です。これをVQEで解くことで、最小値の期待値-8が得られます。

In [5]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

class QubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian, 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
runner = Vqe(QubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-7.986060449670348
Result by numpy
-8.0


正しく得られています。実際にはansatzを作るのが難しかったり、その他定式化に有利な回路を作るためにQAOAを利用するのが無難です。

##2-5 QAOA
VQEはansatzを高度化することで問題特化型のアルゴリズムとして発展をさせることができます。ここでは、VQEの変分法をベースに、ansatzを組合せ最適化問題に特化した形で発展をしたQAOA(Quantum Approximate Opitmization Alogirthm)をみてみます。

##2-5-1 量子断熱計算
量子断熱計算は、状態ベクトルを断続的に変化させることで基底状態をキープしたまま時間発展を行うことができる理論です。

初期状態のハミルトニアンを$H_{start}$として、最終的に求めたい問題のハミルトニアンを$H_{fin}$としたときに、時間$t$と全体のスケジュール$T$から、

$$
H_{temp} = (1-\frac{t}{T})H_{start} + \frac{t}{T}H_{fin}
$$

としたときに$T\rightarrow\infty$とすれば、時間発展で変化させた状態ベクトルが、その瞬間瞬間のハミルトニアンに追従し、固有状態をとり固有値$\lambda$を持つようにすることができます。

$$
H_{temp}\mid \psi \rangle = E_{0temp}\mid \psi \rangle
$$

時間発展計算は、

$$
\mid \psi_{t+1} \rangle = U \mid \psi_t \rangle = e^{-iHt}  \mid \psi_t \rangle
$$

となります。課題は基底状態と第一励起状態が最接近する部分ですが、$E_1(t)-E_0(t)$のエネルギー差に注意して計算することによって、基底状態をキープできます。


##2-5-2 QAOA
QAOAは上記の量子断熱計算の時間発展計算をansatzとして変分アルゴリズムに適用したものです。

![https___qiita-user-contents.imgix.net_https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fe10f1843-cc16-cdfe-e4a6-e2fbaab6df9f.png_ixlib=rb-1.2.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/812819f5-0037-5a30-2690-8ed39c0b1a06.png)

一番左のHは初期の固有状態を作っています。これに対応するハミルトニアンはXです。また、CX-Rz-CXは問題設定のハミルトニアンのウェイトに対応し、次のRzはハミルトニアンのバイアスに対応しています。Rxは上記のハミルトニアンXを時間発展させたものです。

試しにblueqatに用意されたQaoaAnsatzをみてみます。ハミルトニアンは今回Z演算子から構成されており、自動的にハミルトニアンから時間発展のansatzを構成しているのがみて取れます。

一番左のHは初期の固有状態を作っています。これに対応するハミルトニアンはXです。また、CX-Rz-CXは問題設定のハミルトニアンのウェイトに対応し、次のRzはハミルトニアンのバイアスに対応しています。Rxは上記のハミルトニアンXを時間発展させたものです。

試しにblueqatに用意されたQaoaAnsatzをみてみます。ハミルトニアンは今回Z演算子から構成されており、自動的にハミルトニアンから時間発展のansatzを構成しているのがみて取れます。

```python
class QaoaAnsatz(AnsatzBase):
    def __init__(self, hamiltonian, step=1, init_circuit=None):
        super().__init__(hamiltonian, step * 2)
        self.hamiltonian = hamiltonian.to_expr().simplify()
        if not self.check_hamiltonian():
            raise ValueError("Hamiltonian terms are not commutable")

        self.step = step
        self.n_qubits = self.hamiltonian.max_n() + 1
        if init_circuit:
            self.init_circuit = init_circuit
            if init_circuit.n_qubits > self.n_qubits:
                self.n_qubits = init_circuit.n_qubits
        else:
            self.init_circuit = Circuit(self.n_qubits).h[:]
        self.init_circuit.make_cache()
        self.time_evolutions = [term.get_time_evolution() for term in self.hamiltonian]

    def check_hamiltonian(self):
        """Check hamiltonian is commutable. This condition is required for QaoaAnsatz"""
        return self.hamiltonian.is_all_terms_commutable()

    def get_circuit(self, params):
        c = self.init_circuit.copy()
        betas = params[:self.step]
        gammas = params[self.step:]
        for beta, gamma in zip(betas, gammas):
            beta *= np.pi
            gamma *= 2 * np.pi
            for evo in self.time_evolutions:
                evo(c, gamma)
            c.rx(beta)[:]
        return c
```

ライブラリ側で適切な計算をしてくれているので、これ以降は、ほぼSDK側で処理されたQAOAを使った計算をしてみます。今回はQUBO形式でQAOAライブラリを実行します。今回は簡単な定式化を解いてみましょう。

```python
cost = -3*q(0)-3*q(1)-2*q(0)*q(1)
```

明らかにq(0)=1,q(1)=1の時に最小値-3-3-2=-8を取ります。上記のコスト関数は以前のように-1と+1ではなく、0と1で考えることができるので簡単です。

設定するのは、q(0)とq(1)に-3のバイアスを設定し、q(0)*q(1)に-2のウェイトを設定します。


In [6]:
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
step = 2

result = vqe.Vqe(vqe.QaoaAnsatz(h, step)).run()
print(result.most_common(12))

(((1, 1), 0.9552579305295982), ((0, 0), 0.02975250342490814), ((0, 1), 0.0074947830227465335), ((1, 0), 0.007494783022746532))


ハミルトニアンの設定と時間発展の分割を指定しました。回路はちょっと複雑になりました。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fe10f1843-cc16-cdfe-e4a6-e2fbaab6df9f.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=70b8e0d93e68b022e8635d26cd100dbc">

これをQAOAで解くことで、量子ビットが共に1の時、最小値-8が得られます。最初の数字が答えの候補で、次の数字はそれが現れる確率です。(1,1)の組合せが最大確率になっているので正解です。01の量子ビットでの定式化をして、あとはツールに任せればいいというのが分かりました。最初はツールに任せて定式化を頑張りましょう！